# All Together Now
## Fingerprints for many wavelets, clustering, then sorting

Combines the code from 10, 11, 12.  Temporary in Jupyter Notebook, probably going to be converted to .py so it can run headless once it is trustworthy.  Tested on 3 files, going to go to 16.  Will do same stack, 2 cranes before and after tamping, 4 days of each set.

In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import platform

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [2]:
#Wavelet Imports Extra Headers:
import os as os
import statistics as st
import random
import multiprocessing
from joblib import Parallel, delayed
import shutil, glob, os.path
from time import time as ti
from contextlib import contextmanager
import sys
import pandas as pd

 ## Choosing Platform
 Working is beinging conducted on several computers, and author needs to be able to run code on all without rewriting..  This segment of determines which computer is being used, and sets the directories accordingly.

In [3]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"


In [4]:
if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'


In [5]:
if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/Recordings2/'
    imageFolder = '/sciclone/scr10/dchendrickson01/Move3Dprint/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'


## Project specific functions

In [6]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [7]:
def GetResultsDataFrame(folder):
    files = os.listdir(folder)
    if len(files) > 2:
        TypeName = folder.split('/')[-2]
        Results = []
        for file in files:
            Group = int(file.split('_')[0])
            Move = file.split('_')[1][5:-4]
            Results.append([Group,Move])
        Results = np.matrix(Results)
        temp_dict = {
                "MoveName" : np.asarray(Results[:,1]).flatten(),
                TypeName: np.asarray(Results[:,0]).flatten()
            }
        DataSet = pd.DataFrame(temp_dict)
        return DataSet
    

## Set up variables

In [8]:
ClustersWanted = 11
scales= 100
skips = 1
minLength = 750
ImagesIn = imageFolder + 'wvltSort/'

In [9]:
folders = glob.glob(ImagesIn+'*/')


In [10]:
len(folders)


129

## Now for comaprison of the results

In [21]:
RealReturns = False
for folder in folders:
    FolderDF = GetResultsDataFrame(folder)
    #print(folder.split('/')[-2], len(FolderDF))
    if len(FolderDF) > 2:
        if RealReturns == True:
            AllData = pd.merge(AllData, FolderDF, on ='MoveName', how ="outer")
        else:
            AllData = FolderDF
            RealReturns = True

In [22]:
AllData

,MoveName,db12,db17,coif11,coif10,coif12,coif13,db13,db2,db14,...,db1,db3,db4,db5,db6,db7,db8,db9,db10,db11
0,221206 recording1.000,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1,221206 recording1.001,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
2,221206 recording1.002,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
3,221206 recording1.003,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
4,221206 recording1.004,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,230421 recording1.151,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
2187,230421 recording1.152,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
2188,230421 recording1.158,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
2189,230421 recording2.000,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [23]:
AllData.to_csv('ClusteredResults.csv')